# TDM for IGWBS 2: Metadaten-Analyse und Visualisierung

## Was brauchen wir?

In [ ]:
# Benötigte Python Libraries importieren
import pandas as pd                    # Standard Library für Dataframes (tabellenartige Datenstrukturen)
import collections                     # spezielle Daten-Container
import re                              # mit Regex (Regular Expressions) arbeiten
import json                            # mit JSON Daten umgehen (hier: hilfsweise)
import seaborn as sns                  # Visualisierung
import matplotlib.pyplot as plt        # Visualisierung 2
from matplotlib.pyplot import figure
%matplotlib inline
print("Alle Libraries erfolgreich importiert")

In [ ]:
import warnings                        # allfällige Warnungen nicht anzeigen
warnings.filterwarnings('ignore')

## Daten aus CSV in einen Dataframe laden

In [ ]:
infile = 'e-rara_daten_semikolon.csv'
with open(infile, mode='r', encoding='utf-8') as f:
    df = pd.read_csv(f, sep=';')

In [ ]:
df.head(5)

## Einfache Statistiken

In [ ]:
# Überblick über die Daten: Grösse, Spalten, Null-Werte, Datentypen
df.info()

In [ ]:
# wie viele "non-null" Werte sind vorhanden = count
# wie viele unterschiedliche Werte = unique
# häufigste Werte mit Anzahl = top, freq

df.describe(include='object')

In [ ]:
# Wie sehen die 10 häufigsten Werte für das Metadatum "title" aus und wie oft tauchen diese auf?
counter = collections.Counter(df.title)
counter.most_common(10)

In [ ]:
# Analog für "creator"
counter = collections.Counter(df.creator)
counter.most_common(10)

In [ ]:
# Analog für "types"
counter = collections.Counter(df.types)
counter.most_common()

In [ ]:
# Wie sieht es mit den Ausprägungen in "sets" aus, und dies in Anteilen?
df['sets'].value_counts(normalize=True)

In [ ]:
# Alternative: alle Ausprägungen für das Metadatum "language", mit Anzahl
df['language'].value_counts()

In [ ]:
# Wie viele unterschiedliche Datensätze gibt es? Bzw. gibt es Duplikate?
len(df.value_counts(dropna=False))

In [ ]:
# Wie viele Null-Werte (= "Wert nicht-vorhanden") gibt es?
df.isnull().sum().sum()

In [ ]:
# Und in einer spezifischen Spalte, hier "subjects"?
df.subjects.isna().sum()

In [ ]:
# In allen Spalten im Überblick
df.isna().sum()

In [ ]:
# Alle Zeilen mit mindestens einem Null-Wert.
df[df.isna().any(axis=1)]

In [ ]:
# Gegenüberstellung zweier Werte
pd.crosstab(df.language, df.types)

In [ ]:
# Variante
df.groupby(['language', 'types']).size()

## Datentransformation und Bereinigung

### Sets, Subjects und Types auswerten

In [ ]:
# Problem: Es sind mehrere Werte in den "sets"-Feldern enthalten - allerdings nicht als Liste, sondern als String
# -> macht Auswertung schwer

print(df.sets[721])
print('---')
print(type(df.sets[721]))

In [ ]:
# Dasselbe gilt für die "subjects" und "Types"
print(df.subjects[721])
print('---')
print(type(df.subjects[721]))
print('---')
print(df.types[721])
print('---')
print(type(df.types[721]))

In [ ]:
# Drei neue Spalten "sets_list", "types_list" und "subjects_list", in der Sets, Types und Subjects wieder zählbare Listen sind!

df['sets_list'] = 0
for i in df.index:
    string = df.sets[i]
    string2 = string.replace('\'', '"')
    df['sets_list'][i] = json.loads(string2) 
    
df['types_list'] = 0
for i in df.index:
    string = df.types[i]
    string2 = string.replace('\'', '"')
    df['types_list'][i] = json.loads(string2) 
    
df['subjects_list'] = 0
for i in df.index:
    try:
        #print(type(df['subjects'][i]))
        string = df.subjects[i]
        string2 = string.replace('\'', '"')
        df['subjects_list'][i] = json.loads(string2)
    except: pass

In [ ]:
df[670:675]

In [ ]:
# Gegenprobe
print(type(df.sets_list[3]))
print(type(df.types_list[3]))
print(type(df.subjects_list[3]))

In [ ]:
# Auslesen allen Listenelmente in der gesamte Spalte "sets_list" und zusammenfassen in einer externen Gesamtliste "sets_all"
sets_all = []
for i in df.index:
    sets_all.extend(df.sets_list[i])
    
len(sets_all)

In [ ]:
# Auswertung von "sets_all"
counter = collections.Counter(sets_all)
counter.most_common(20)

In [ ]:
# Auslesen allen Listenelmente in der gesamte Spalte "sets_list" und zusammenfassen in einer externen Gesamtliste "sets_all"
types_all = []
for i in df.index:
    types_all.extend(df.types_list[i])
    
len(types_all)

In [ ]:
# Auswerten von "types_all"
counter = collections.Counter(types_all)
counter.most_common()

In [ ]:
# Analog für "subjects_list" -> externe GesamtListe "subjects_all" zur Auswertung
subjects_all = []
for i in df.index:
    if type(df.subjects_list[i]) == list:
        subjects_all.extend(df.subjects_list[i])
        
len(subjects_all)

In [ ]:
# Auswertung von "subjects_all"
counter = collections.Counter(subjects_all)
counter.most_common(20)

### Start-Jahre auslesen

In [ ]:
# Eine neue Spalte bilden mit dem einzigen bzw. erstgenannten Jahr = "start_year"

df['start_year'] = 0
for i in df.index:
    match = re.search('(\d+)', df.year[i])
    if match:
        y = match.group(1)
    else:
        y = 0
    df['start_year'][i] = y
df.start_year.astype('int')

In [ ]:
# alternative Anzeige der ersten 5 Zeilen zu df.head(5)
df[:5]

In [ ]:
# Datentypen in den Spalten: Es gibt einen neuen numerischen Datentyp, "int64".
df.dtypes

In [ ]:
# statistischer Überblick über numerische Variablen 
# Jahreszahl Minimum "0"? Da stimmt noch etwas nicht

df.describe().round(0)

In [ ]:
# Jahreszahl-Minimum "0" - was hat es damit auf sich? Anzeige dieser Datensätze...
df[df.start_year==0]

In [ ]:
# Weitere Überprüfung: Zeige alle Startjahre, die kleiner sind als 1800 ODER weniger Stellen haben als 4
# -> entsprechend auskommentieren

for i in df.index:
    #if df.start_year[i] < 1800:
    if len(str(df.start_year[i])) < 4:
        print(df.start_year[i])

In [ ]:
# Auffüllen der Jahreszahlen mit weniger als 4 Stellen mit Nullen
for i in df.index:
    if len(str(df.start_year[i])) < 4:
        n = 4 - len(str(df.start_year[i]))
        df.start_year[i] = str(df.start_year[i]) + n * (str(0))
df.start_year.astype('int')

In [ ]:
# Gegenprobe: Nochmal alle Jahreszahlen < 1800 oder mit weniger als 4 Stellen

for i in df.index:
    if df.start_year[i] < 1800:
    #if len(str(df.start_year[i])) < 4:
        print(df.start_year[i])

In [ ]:
# Die "0" stört aber weiterhin, z.B. bei statistischen Auswertungen, daher:
# Transformation der "0" in einen richtigen Null-Wert, d.h. ein "Nicht-Vorhanden-Wert"

for i in df.index:
    if df.start_year[i] == 0:
        df.start_year[i] = None

In [ ]:
# Nun sieht die statistische Auswertung für "start_year" valide aus.
df.describe().round()

In [ ]:
# Wie sehen die 10 häufigsten Werte für "start_year" nun aus und wie oft tauchen diese auf?
counter = collections.Counter(df.start_year)
counter.most_common(10)

## Visualisierungen

### Visualisierungen direkt aus Pandas

In [ ]:
# Streudiagramm
ax = df.plot(x='start_year', y='language', kind='scatter', s=2.5, figsize=(8,4))

In [ ]:
# Balkendiagramm für "language"
figure(figsize=(6, 6), dpi=80)
ax = df.language.value_counts().plot(kind='bar', color='darkred')

In [ ]:
# Tortendiagramm
figure(figsize=(6, 6), dpi=80)
ax = df.language.value_counts().plot(kind='pie')    

In [ ]:
# Histogramm (hier: 10 Kohorten)
ax = df.start_year.plot(kind='hist', bins=20, color='lightgreen')                                

In [ ]:
# Kasten-Diagramm (box whisker plot)
ax = df.start_year.plot(kind='box', vert=False)

In [ ]:
# Dichte-Diagramm
ax = df.start_year.plot(kind='density')

In [ ]:
# Gruppiertes Kastendiagramm 
figure(figsize=(10, 10), dpi=80)
ax = df.boxplot(column='start_year', by='language', vert=False, grid=False, rot=30, fontsize=14)

### Visualisierungen mit Seaborn

In [ ]:
# Gruppiertes Kastendiagramm mit Seaborn
figure(figsize=(8, 6), dpi=80)
ax = sns.boxplot(y='types', x='start_year', data=df, orient='h', palette='Set3')

In [ ]:
# Heatmap mit Seaborn: Types vs. Language
ct = pd.crosstab( df['types'], df['language'])
ax = sns.heatmap(ct, annot=True, fmt='d', cmap='magma_r', linewidths=.5)

### Visualisierungen mit Matplotlib

In [ ]:
# Visualisierung von "sets_all"
counter = collections.Counter(sets_all)
sets = counter.most_common(20)
x, y = zip(*sets)

plt.barh(x, y, color='violet')
plt.show()

In [ ]:
# Visualisierung von "subjects_all"
counter = collections.Counter(subjects_all)
subjects = counter.most_common(10)
x, y = zip(*subjects)

plt.barh(x, y, color='teal')
plt.show()

In [ ]:
# Visualisierung von "types"
counter = collections.Counter(types_all)
types = counter.most_common(20)
x, y = zip(*types)

plt.barh(x, y, color='gold')
plt.show()